In [10]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

In [11]:
dataset_path= '../data/tiles/formatted'

In [6]:
class ImageDataset(Dataset):
    def __init__(self, dataset_path) -> None:
        files = os.listdir(dataset_path)
        self.images_path = []
        self.masks_path = []
        for file in files:
            if file.split('_')[1]=='tif':
                self.images_path.append(os.path.join(dataset_path,file))
                self.masks_path.append(os.path.join(dataset_path,file.replace('tif','shp')))
                
    def __len__(self):
        return len(self.images_path)
    
    def __getitem__(self, idx):
        return {
            'image': self.images_path[idx],
            'mask': self.masks_path[idx]
        }

In [12]:
dataset = ImageDataset(dataset_path)
data_loader = DataLoader(dataset, batch_size=1)

for batch in data_loader:
    print(batch)

{'image': ['../data/tiles/formatted/tile_tif_0_2.png'], 'mask': ['../data/tiles/formatted/tile_shp_0_2.png']}
{'image': ['../data/tiles/formatted/tile_tif_2_0.png'], 'mask': ['../data/tiles/formatted/tile_shp_2_0.png']}
{'image': ['../data/tiles/formatted/tile_tif_2_1.png'], 'mask': ['../data/tiles/formatted/tile_shp_2_1.png']}
{'image': ['../data/tiles/formatted/tile_tif_0_3.png'], 'mask': ['../data/tiles/formatted/tile_shp_0_3.png']}
{'image': ['../data/tiles/formatted/tile_tif_0_1.png'], 'mask': ['../data/tiles/formatted/tile_shp_0_1.png']}
{'image': ['../data/tiles/formatted/tile_tif_2_3.png'], 'mask': ['../data/tiles/formatted/tile_shp_2_3.png']}
{'image': ['../data/tiles/formatted/tile_tif_2_2.png'], 'mask': ['../data/tiles/formatted/tile_shp_2_2.png']}
{'image': ['../data/tiles/formatted/tile_tif_0_0.png'], 'mask': ['../data/tiles/formatted/tile_shp_0_0.png']}
{'image': ['../data/tiles/formatted/tile_tif_0_4.png'], 'mask': ['../data/tiles/formatted/tile_shp_0_4.png']}
{'image': 